# Pilot 1 -- Compute the baseline decodability of Motor rule response (LINDEX v. LMID and RINDEX v. RMID)

## Use SVM classifications to decode hand-specific responses

## Takuya Ito
#### 02/23/2018

In [14]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [15]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.0 Define functions for loading data

In [17]:
def loadMotorResponses(subj):
    datadir = basedir + 'data/results/GLM_MotorResponse/' 
    data = np.loadtxt(datadir + subj + '_motorResponse_taskBetas_Surface64k_GSR.csv',delimiter=',')
    data = data[:,-4:]
    return data

# def loadRestFC(subj,fctype='pearson',gsr=True):
#     datadir = '/projects3/NetworkDiversity/data/hcppreprocessed/' + subj + '/' 
#     if fctype=='pearson':
#         if gsr:
#             data = np.loadtxt(datadir + subj + '_rest1_nuisanceResids_Glasser_GSR.csv',delimiter=',')
#         else:
#             data = np.loadtxt(datadir + subj + '_rest1_nuisanceResids_Glasser.csv',delimiter=',')
#         fc = np.corrcoef(data)
#     elif fctype=='multregfc':
#         if gsr:
#             direc = '/projects3/NetworkDiversity/data/results/multRegRestFC_GlasserParcels_HCPData/' 
#             fc = np.loadtxt(direc + subj + '_multreg_restfc_gsr.txt', delimiter=',')
#         else:
#             direc = '/projects3/NetworkDiversity/data/results/multRegRestFC_GlasserParcels_HCPData/' 
#             fc = np.loadtxt(direc + subj + '_multreg_restfc.txt', delimiter=',')
# #             fc = np.loadtxt(direc + subj + '_OutNetToFPN_multreg_restfc.txt', delimiter=',')
#     return fc
        

## 0.1 Load data

In [18]:
# gsr = True
nResponses = 4
data_task = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadMotorResponses(subj)
    scount += 1

# 1.0 Define functions for motor response decodings

In [19]:
from scipy.spatial.distance import cdist

def motorResponseDecodings(data, ncvs=100, nproc=5):
    """
    Run an across-subject classification
    Decode responses on each hand separately from CPRO data
    Limit to ROIs within SMN network
    """
    
    hands = {'Left':[0,1],'Right':[2,3]}
    
    nSubjs = data.shape[2]
    nHands = len(hands)
    rois = np.where(networkdef==networkmappings['smn'])[0] + 1
    smnStats = np.zeros((len(rois),nSubjs,nHands))

    taskcount = 0
    for hand in hands:
        print 'Computing SVM classification for task', hand

        nfing = len(hands[hand])
        hand_ind = hands[hand]
        
        nsamples = nSubjs * nfing

        # Label array for supervised learning
        labels = np.tile(range(nfing),nSubjs)
        subjarray = np.repeat(range(nSubjs),nfing)

        # Run SVM classifications on network-level activation patterns across subjects
        roicount = 0
        for roi in rois:
            roi_ind = np.where(glasser2==roi)[0]
            nfeatures = len(roi_ind)
            roi_ind.shape = (len(roi_ind),1)       

            svm_mat = np.zeros((nsamples,roi_ind.shape[0]))
            samplecount = 0
            scount = 0
            for subj in range(len(subjNums)):
                roidata = np.squeeze(data[roi_ind,hand_ind,scount])
                svm_mat[samplecount:(samplecount+nfing),:] = roidata.T

                scount += 1
                samplecount += nfing

#             # Spatially demean matrix across features
#             samplemean = np.mean(svm_mat,axis=1)
#             samplemean.shape = (len(samplemean),1)
#             svm_mat = svm_mat - samplemean
            
            scores = randomSplitLOOBaselineCV(ncvs, svm_mat, labels, subjarray, nproc=nproc)
            smnStats[roicount,:,taskcount] = scores
            roicount += 1
        
        taskcount += 1

    return smnStats

def randomSplitLOOBaselineCV(ncvs, svm_mat, labels, subjarray, nproc=5):
    """
    Runs cross validation for an across-subject SVM analysis
    """
    
    ntasks = len(np.unique(labels))
    nsamples = svm_mat.shape[0]
    nsubjs = nsamples/ntasks

    subjects = np.unique(subjarray)
    indices = np.arange(nsamples)

    inputs = [] 
    for subj in subjects:
        test_subj = subj
        train_subjs_all = np.delete(subjects,test_subj)
        for cv in range(ncvs):
            # Randomly sample half of train set subjects for each cv (CV bootstrapping)
            test_subjs = np.random.choice(train_subjs_all,
                                         int(np.floor(len(train_subjs_all)/2.0)),
                                         replace=True)
            # Include all subjs not in train subjs into test subjs for mahalanobis computation
            test_subjs = np.hstack((test_subj,test_subjs))
            train_subjs = np.delete(subjects,test_subjs)

            train_ind = []
            for subj in train_subjs:
                train_ind.extend(np.where(subjarray==subj)[0])

            test_ind = []
            for subj in test_subjs:
                if subj == test_subj:
                    # Keep track of how many samples this subject has
                    subj_samples = len(np.where(subjarray==subj)[0]) 
                test_ind.extend(np.where(subjarray==subj)[0])
            
            train_ind = np.asarray(train_ind)
            test_ind = np.asarray(test_ind)

            trainset = svm_mat[train_ind,:]
            testset = svm_mat[test_ind,:]

            # Normalize trainset and testset
            mean = np.mean(svm_mat[train_ind,:],axis=0)
            mean.shape = (1,len(mean))
            std = np.std(svm_mat[train_ind,:],axis=0)
            std.shape = (1,len(std))

            trainset = np.divide((trainset - mean),std)
            testset = np.divide((testset - mean),std)

            inputs.append((trainset,testset,labels[train_ind],labels[test_ind],subj_samples))
        
    pool = mp.Pool(processes=nproc)
    scores = pool.map_async(_decoding,inputs).get()
    pool.close()
    pool.join()
    
    subj_acc = np.zeros((len(subjects),))
    scount = 0
    i = 0
    for subj in subjects:
        subjmean = []
        for cv in range(ncvs):
            subjmean.append(scores[i])
            i += 1
        
        subj_acc[scount] = np.mean(subjmean)
        
        scount += 1

    return subj_acc

def _decoding((trainset,testset,trainlabels,testlabels,subj_samples)):

#     clf = sklearn.linear_model.LogisticRegression()
    clf = svm.SVC(C=1.0, kernel='linear')

    clf.fit(trainset,trainlabels)
    predictions = clf.predict(testset)
    acc = predictions==testlabels
    score = np.mean(acc[0:subj_samples])
    
    return score

    

## 1.1 Run across subject decoding on hand-specific motor responses

In [20]:
nproc = 25
ncvs = 50

distances_baseline = motorResponseDecodings(data_task, ncvs=ncvs, nproc=nproc)


Computing SVM classification for task Right
Computing SVM classification for task Left


## 1.2 Compute statistics

In [21]:
hands = {'Left':[0,1],'Right':[2,3]}
smnROIs = np.where(networkdef==networkmappings['smn'])[0] + 1

statistics = {}
taskcount = 0
for hand in hands:
    statistics[hand] = np.zeros((len(smnROIs),4)) # acc, t, p, q
    for roicount in range(len(smnROIs)):
        t, p = stats.ttest_1samp(distances_baseline[roicount,:,taskcount],1/float(len(hands[hand])))
        if t > 0:
            p = p/2.0
        else:
            p = 1.0 - p/2.0
        
        statistics[hand][roicount,0] = np.mean(distances_baseline[roicount,:,taskcount])
        statistics[hand][roicount,1] = t
        statistics[hand][roicount,2] = p
        
    qs = mc.fdrcorrection0(statistics[hand][:,2])[1]
    for roicount in range(len(smnROIs)):
        statistics[hand][roicount,3] = qs[roicount]
    
    taskcount += 1
    

In [22]:
# Count number of significant ROIs for LH decoding
sigLH_ind = np.where(statistics['Left'][:,3]<0.05)[0]
print 'Number of ROIs significant for left hand responses:', sigLH_ind.shape[0]
print 'Accuracies:', statistics['Left'][sigLH_ind,0]

print ''
# Count number of significant ROIs for RH decoding
sigRH_ind = np.where(statistics['Right'][:,3]<0.05)[0]
print 'Number of ROIs significant for right hand responses:', sigRH_ind.shape[0]
print 'Accuracies:', statistics['Right'][sigRH_ind,0]

Number of ROIs significant for left hand responses: 5
Accuracies: [ 0.57333333  0.69447917  0.5546875   0.55416667  0.55020833]

Number of ROIs significant for right hand responses: 7
Accuracies: [ 0.57729167  0.67        0.58322917  0.580625    0.54322917  0.53677083
  0.57333333]


## 1.3 Map accuracies back to cortical surface

In [23]:
# Put all data into a single matrix (since we only run a single classification)
lefthand = np.zeros((glasser2.shape[0],4))
righthand = np.zeros((glasser2.shape[0],4))

lh_ind = [0,1]
rh_ind = [0,1]
roicount = 0
for roi in smnROIs:
    vertex_ind = np.where(glasser2==roi)[0]
    lefthand[vertex_ind,0] = statistics['Left'][roicount,0]
    lefthand[vertex_ind,1] = statistics['Left'][roicount,1]
    lefthand[vertex_ind,2] = statistics['Left'][roicount,2]
    if statistics['Left'][roicount,3] < 0.05: lefthand[vertex_ind,3] = statistics['Left'][roicount,0]

    righthand[vertex_ind,0] = statistics['Right'][roicount,0]
    righthand[vertex_ind,1] = statistics['Right'][roicount,1]
    righthand[vertex_ind,2] = statistics['Right'][roicount,2]
    if statistics['Right'][roicount,3] < 0.05: righthand[vertex_ind,3] = statistics['Right'][roicount,0]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_MotorResponse/'
filename = 'smnDecodingsLH'
np.savetxt(outdir + filename + '.csv', lefthand,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)

outdir = '/projects3/SRActFlow/data/results/Decoding_MotorResponse/'
filename = 'smnDecodingsRH'
np.savetxt(outdir + filename + '.csv', righthand,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0